# MQTT brokers

1. Let us see how to setup an MQTT broker and send messages 
2. Now, try yourself: Use the diabetes.csv dataset to do the following: 
    1. Select the following 4 attributes (3 features + 1 class label) : • Glucose, BloodPressure, Insulin, Outcome 
    2. Normalize Glucose, BloodPressure and Insulin to [0, 1] using MinMax. 
    3. Store the new data (3 normalized features + 1 class label) in another dataset S. 
    4. Modify the MQTT example to do the following: • The publisher publishes records in S continuously. When it reaches the end of S, it continues to send from the beginning again. • The subscriber continuously receives the data. For each latest record r received, apply the 3NN classification to the last 5 records before r, and compare the classification result with the Outcome label in r.
    Repeat this for 1000 times, and report the number of correct classifications

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler()

1. Let us see how to setup an MQTT broker and send messages 
2. Now, try yourself: Use the diabetes.csv dataset to do the following: 
    1. Select the following 4 attributes (3 features + 1 class label) : • Glucose, BloodPressure, Insulin, Outcome 
    2. Normalize Glucose, BloodPressure and Insulin to [0, 1] using MinMax. 
    3. Store the new data (3 normalized features + 1 class label) in another dataset S. 
    



In [36]:
diabetes = pd.read_csv('diabetes.csv')

s_diabetes = diabetes[['Glucose', 'BloodPressure', 'Insulin', 'Outcome']]
s_diabetes.head()

minMaxScaler = MinMaxScaler()
S = pd.DataFrame(minMaxScaler.fit_transform(s_diabetes), columns=s_diabetes.columns)

S

,Glucose,BloodPressure,Insulin,Outcome
0,0.743719,0.590164,0.000000,1.0
1,0.427136,0.540984,0.000000,0.0
2,0.919598,0.524590,0.000000,1.0
3,0.447236,0.540984,0.111111,0.0
4,0.688442,0.327869,0.198582,1.0
...,...,...,...,...
763,0.507538,0.622951,0.212766,0.0
764,0.613065,0.573770,0.000000,0.0
765,0.608040,0.590164,0.132388,0.0
766,0.633166,0.491803,0.000000,1.0


 4. Modify the MQTT example to do the following: 
 - The publisher publishes records in S continuously. When it reaches the end of S, it continues to send from the beginning again. 
 - The subscriber continuously receives the data. For each latest record r received, apply the 3NN classification to the last 5 records before r, and compare the classification result with the Outcome label in r.

In [37]:
import time
import paho.mqtt.client as mqtt
import json

# We create a client as the data publisher
mqttc = mqtt.Client()
# We are connecting to port 1883 and set *keepalive* parameter to 60; *keepalive* is the maximum waiting time in seconds.
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# We send the following readings repeatedly. Once we reach the end, we start from the beginning again.
period = len(S)

if __name__ == "__main__":
    print("Publishing...")
    
    index = 0
    while (index >= 0):
        # Get the current data reading to send out
        diabetes = S.iloc[index % period]
        
        #Publisher is open to all! Make sure use your own ‘Data name’ in your exercise of Publisher/Subscriber
        mqttc.publish("myroom/diabetes", json.dumps(diabetes.to_dict()))
        
        # We send the next reading after 2 seconds
        time.sleep(2)
        index = index + 1

Publishing...


KeyboardInterrupt: 